In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from collections import Counter

In [173]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.98


In [4]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [174]:
#store gap values
Y_train = df_train.gap.values

# Take a sample with training and validation sets
df_sample = df_train.sample(300000, random_state = 42)
msk = np.random.rand(len(df_sample)) < 0.8

zero_cols = []
for i in range(1,257):
    if df_sample['feat_%03d' % i].sum() == 0:
        zero_cols.append('feat_%03d' % i)
df_sample = df_sample.drop(zero_cols, axis=1)

In [175]:
# Add smile character counts
smiles = df_sample.smiles
smileydict = smiles.map(lambda x: dict(Counter(x)))
smile_alphabet=list(set(''.join(smiles.iloc[0:50])))
for smile in smile_alphabet:
    smilechar = smile
    if smile == '=':
        smilechar = 'equal'
    df_sample['smile_'+smilechar] = smileydict.map(lambda x: x[smile] if smile in x.keys() else 0)

In [176]:
# Feature Engineering
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)

# Add length of smile
df_sample['smile_length'] = df_sample.smiles.map(lambda x: len(x))

# Add number of C's divided by length
df_sample['smile_percentc'] = (df_sample.smile_c / df_sample.smile_length)
df_sample['smile_percentC'] = (df_sample.smile_C / df_sample.smile_length)

# Count specific molecules
# [nH]
df_sample['smile_nh'] = df_sample.smiles.map(lambda x: x.count('[nH]'))
df_sample['smile_si'] = df_sample.smiles.map(lambda x: x.count('Si'))
df_sample['smile_sih2'] = df_sample.smiles.map(lambda x: x.count('[SiH2]'))
df_sample['smile_CdoubleC'] = df_sample.smiles.map(lambda x: x.count('C=C'))
df_sample['smile_doubleC'] = df_sample.smiles.map(lambda x: x.count('CC'))
df_sample['smile_doublec'] = df_sample.smiles.map(lambda x: x.count('cc'))
df_sample['smile_triplec'] = df_sample.smiles.map(lambda x: x.count('ccc'))
df_sample['smile_quadc'] = df_sample.smiles.map(lambda x: x.count('cccc'))
df_sample['smile_C1equalCc2'] = df_sample.smiles.map(lambda x: x.count('C1=Cc2'))

# Start
df_sample['smile_startC1'] = df_sample.smiles.map(lambda x: x.startswith('C1'))
df_sample['smile_startc1'] = df_sample.smiles.map(lambda x: x.startswith('c1'))
df_sample['smile_startnH'] = df_sample.smiles.map(lambda x: x.startswith('[nH]'))


#df_sample['smile_percent_bond'] = df_sample.smile_equal / df_sample.smile_length

df_sample = df_sample.drop('smile_length', axis=1)

In [167]:
df_sample.head(20)

,smiles,feat_001,feat_005,feat_006,feat_007,feat_025,feat_037,feat_044,feat_068,feat_069,...,smile_CdoubleC,smile_doubleC,smile_doublec,smile_triplec,smile_quadc,smile_C1equalCc2,smile_startC1,smile_startc1,smile_startnH,smile_si
987231,[nH]1ccc2c3nsnc3c3c(sc4cc(-c5ccc[se]5)c5cscc5c...,0,1,1,1,0,0,0,0,0,...,0,0,4,2,0,0,False,False,True,0
79954,c1cc2c3cocc3c3c4CC(=Cc4oc3c2o1)c1nccs1,1,1,0,1,0,1,0,0,1,...,0,1,3,0,0,0,False,True,False,0
567130,C1=Cc2c(C1)c1ccc3cc(cnc3c1c1c[nH]cc21)-c1scc2s...,1,1,1,1,0,1,0,1,1,...,0,0,5,2,0,1,True,False,False,0
500891,c1cnc(s1)-c1sc(-c2ncc(s2)-c2cccc3c[nH]cc23)c2s...,0,1,1,1,0,0,0,1,0,...,0,0,5,2,1,0,False,True,False,0
55399,C1=CC=C([SiH2]1)C1=Cc2oc3ccc4cscc4c3c2C1,1,1,0,1,1,1,0,0,1,...,1,1,2,1,0,1,True,False,False,1
135049,c1Cc(cc1)-c1ccc([se]1)-c1ccc(-c2ncncn2)c2c[SiH...,1,1,0,1,1,0,0,0,0,...,0,0,4,2,0,0,False,True,False,1
733378,c1ccc2c(c1)c1cscc1c1c3oc(cc3cnc21)-c1cccc2=CCC...,1,1,0,1,1,0,0,1,0,...,0,1,5,2,1,0,False,True,False,0
732057,c1sc(-c2[SiH2]c(cc2)-c2Cc(cc2)-c2cccnc2)c2ccCc12,1,1,0,1,1,1,0,0,1,...,0,0,4,1,0,0,False,True,False,1
51333,c1ccc(o1)-c1ccc(o1)-c1ccc(s1)-c1ccc[se]1,0,1,0,1,0,0,0,1,0,...,0,0,4,4,0,0,False,True,False,0
731479,C1=Cc2c(C1)c1[se]c3cc(sc3c1c1nsnc21)-c1scc2ccsc12,1,1,0,1,0,1,0,1,1,...,0,0,3,0,0,1,True,False,False,0


In [177]:
df_sample_train = df_sample[msk]
Y_sample_train = df_sample_train.gap.values
df_sample_train = df_sample_train.drop(['gap'], axis=1)

df_sample_test = df_sample[~msk]
Y_sample_test = df_sample_test.gap.values
df_sample_test = df_sample_test.drop(['gap'], axis=1)

print "Training Sample Shape", df_sample_train.shape
print "Y Training Sample Shape", Y_sample_train.shape
print "Test Sample Shape", df_sample_test.shape
print "Y Test Sample Shape", Y_sample_test.shape

Training Sample Shape (240269, 67)
Y Training Sample Shape (240269,)
Test Sample Shape (59731, 67)
Y Test Sample Shape (59731,)


In [130]:
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

ValueError: labels ['Id'] not contained in axis

In [8]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [178]:
#Drop the 'smiles' column
# df_all = df_all.drop(['smiles'], axis=1)
# vals = df_all.values
# X_train = vals[:test_idx]
# X_test = vals[test_idx:]
# print "Train features:", X_train.shape
# print "Train gap:", Y_train.shape
# print "Test features:", X_test.shape

df_sample_train = df_sample_train.drop(['smiles'], axis=1)
df_sample_test = df_sample_test.drop(['smiles'], axis=1)

In [179]:
# Sample Linear Regression Testing
LR = LinearRegression()

# Train the model using the training sets
LR.fit(df_sample_train, Y_sample_train)

# The coefficients
#print('Coefficients: \n', LR.coef_)
# RMSE
print("RMSE: %.5f" % np.sqrt(np.mean((LR.predict(df_sample_test) - Y_sample_test) ** 2)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % LR.score(df_sample_test, Y_sample_test))

RMSE: 0.21192
Variance score: 0.73


In [31]:
# Sample Polynomial Interpolation Testing
degreearray=[]
scorearray=[]

for degree in range(2, 4):
    degreearray.append(degree)
    model = make_pipeline(PolynomialFeatures(degree), Ridge())
    model.fit(df_sample_train, Y_sample_train)
    score = np.sqrt(np.mean((model.predict(df_sample_test) - Y_sample_test) ** 2))
    scorearray.append(score)
    print "Degree %d RMSE: %f" % (degree, score)

Degree 2 RMSE: 0.282621


KeyboardInterrupt: 

In [180]:
num_trees = [40]
for trees in num_trees:
    RF = RandomForestRegressor(n_estimators=trees, min_samples_split=10)
    RF.fit(df_sample_train, Y_sample_train)
    print np.sqrt(np.mean((RF.predict(df_sample_test) - Y_sample_test) ** 2))

0.137673991488


In [40]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [48]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)